### 1. 🧪 Install Required Libraries (Run only once)

In [1]:


# !pip install -q langchain faiss-cpu python-dotenv pypdf unstructured huggingface_hub transformers

# Optional: For local models with Ollama (skip if using Hugging Face only)
# !pip install langchain-community


### 2. 🔐 Load Environment Variables (from .env)

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")


### 3. 📚 Load Your LOTR PDF (Adjust path as needed)

In [3]:


from langchain.document_loaders import PyPDFLoader

pdf_path = "Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

print(f"Loaded {len(pages)} pages from PDF.")


Loaded 1210 pages from PDF.


### 4. ✂️ Chunk the Text for Embedding

In [4]:


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(pages)
print(f"Total chunks: {len(docs)}")


Total chunks: 4246


### 5. 🔎 Create FAISS Vector Store with Embeddings

In [5]:
# !pip install -U langchain-huggingface sentence-transformers

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
import os

load_dotenv()
# Optional for safety/flexibility
# os.environ["HUGGINGFACE_HUB_TOKEN"] = os.getenv("HUGGINGFACE_HUB_TOKEN")
'''
may need to downgrade keras to 2.11.0
pip uninstall keras -y
pip install keras==2.11.0
'''
# ✅ This is all you need
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("gandalf_index")

print("✅ Vectorstore saved.")


✅ Vectorstore saved.


### 6. 🤖 Load a Local LLM (Option 1: Hugging Face)

In [7]:
# pip install -U langchain-huggingface

In [10]:
from langchain.llms import HuggingFaceHub
from langchain_huggingface import HuggingFaceEndpoint

from dotenv import load_dotenv
import os



load_dotenv()
hf_token = os.getenv("HUGGINGFACE_HUB_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="tiiuae/falcon-7b-instruct",
    temperature=0.7,
    max_new_tokens=512,
    huggingfacehub_api_token=hf_token
)


# OPTIONAL: 6B 🤖 Load from Ollama (Option 2: Local Mistral/WizardLM)

In [11]:


# from langchain_community.llms import Ollama
# llm = Ollama(model="mistral")  # or wizardlm


In [12]:
### 7. 🔁 Ask Gandalf a Question (RAG Chain)

from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

question = "What happened in the mines of Moria?"
result = qa_chain(question)

print("🧙 Gandalf says:\n", result['result'])


C:\Users\JonMa\AppData\Local\Temp\ipykernel_35800\1316374485.py:14: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(question)
C:\Users\JonMa\AppData\Roaming\Python\Python313\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


🧙 Gandalf says:
  The dwarves were trapped in the depths of Moria, and the Orcs had been mining the ore from the mountains for centuries. The dwarves were enslaved by the Orcs and forced to work in the mines. They had to endure horrible conditions, including dangerous gas and poisonous fumes. Gandalf and the others had to help the dwarves escape from Moria.
